# Main Script

In this project, bounding boxes are connected to multiple frames using the Hungarian Algorithm!

This work is done based on 4 aspects of the **multi-object tracker**:

*   Using YOLO and launch an object detection algorithm
*   Using The Hungarian Algorithm and connecting the boxes
*   Improving the algorithm to prevent false positives and false negatives
*   Using Kalman Filters to help predict the future position of a bounding box

This is a part included to link the Google Colab file (.ipynb) to your Google Drive folder.

In [1]:
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=False)
os.chdir("/content/drive/My Drive/Colab Notebooks/Tracking")
!ls

Mounted at /content/drive
Association  Detection	Main  movie.mp4  README.md


# 1 - Detection

For Tracking, the first step is detection. The tracking heavily relies on the detection. In this project, [YOLO algorithm](https://pjreddie.com/darknet/yolo/) was selected because of accuracy and speed.

<img src="https://miro.medium.com/max/1446/1*YpNE9OQeshABhBgjyEXlLA.png" width="500">

Eventually, our aim is to have a bounding box detection like the below picture.

<img src="https://pjreddie.com/media/image/Screen_Shot_2018-03-24_at_10.48.42_PM.png" width="500">


## Import Libraries and Test Images

In this project, the frame per second of video is sixty fps but we use every seven image in fps. In other words, Instead of working at **60 FPS** (recording frame rate), the algorithm is working at 60/7 or about **9 frame per second**.<p>
**The reason behind this ?**<p>
YOLO is very fast, it can work at 60 FPS. For tracking purpose, 60 fps is a bit challenging. Because of that, let's not have 99% IOU every time.

In [2]:
### Imports
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import copy
import pickle
import cv2

In [3]:
### Load the Images
dataset_images = pickle.load(open('Main/Images/images_tracking.p', "rb"))

In [4]:
def visualize_images(input_images):
    fig=plt.figure(figsize=(100,100))

    for i in range(len(input_images)):
        fig.add_subplot(1, len(input_images), i+1)
        plt.imshow(input_images[i])
    plt.show()

In [5]:
visualize_images(dataset_images)

Output hidden; open in https://colab.research.google.com to view.

In [6]:
os.chdir("/content/drive/My Drive/Colab Notebooks/Tracking/Main")
### Run obstacle detection for the images
from yolo_for_tracking import *

result_images = [] # Empty list for output images
result_boxes = [] # Empty list for output boxes

# Initiliaze an object detector
detector = YOLO()

images = copy.deepcopy(dataset_images)

# For every image, run a detector using the inference() function
for img in images:
    result, boxes = detector.inference(img)
    result_images.append(result)
    result_boxes.append(boxes)

print(result_boxes)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[[[474, 207, 295, 158], [147, 213, 74, 51], [265, 214, 65, 53], [112, 212, 12, 22], [346, 203, 42, 43], [216, 214, 38, 31], [149, 215, 73, 50], [265, 221, 65, 43]], [[469, 208, 302, 155], [143, 213, 73, 51], [259, 214, 68, 53], [103, 213, 13, 22], [253, 216, 16, 12], [339, 202, 43, 45], [145, 210, 73, 49], [212, 216, 40, 30], [262, 222, 63, 43]], [[471, 206, 296, 155], [148, 212, 72, 51], [264, 211, 69, 56], [344, 202, 44, 45], [654, 205, 18, 24], [153, 209, 68, 52], [217, 213, 39, 30], [265, 220, 65, 43]], [[471, 214, 293, 146], [153, 214, 68, 51], [269, 213, 64, 57], [102, 212, 13, 25], [681, 204, 21, 39], [218, 217, 42, 29], [342, 209, 51, 42], [425, 216, 11, 25], [711, 213, 19, 43], [157, 218, 64, 46], [269, 222, 63, 44]], [[477, 211, 283, 143], [158, 213, 72, 52], [273, 215, 70, 50], [352, 204, 43, 41], [395, 213, 9, 18], [425, 211, 9, 19], [446, 211, 11

In [7]:
# Print the results and the detected boxes
visualize_images(result_images)

Output hidden; open in https://colab.research.google.com to view.

## For each Obstacle, specific Color is assigned
**Last Step!** <p>
All cars in one color is useless! Because of that, the code will be modified <p>
Each detected obstacle has:
* an id
* a current bounding box
* a previous bounding box<p>_

**In the end, we will draw a bounding box based on the id.** <p>
If the id changes, the color will change as well.

In [8]:
class Obstacle():
    def __init__(self, idx, box):
        """
        Init function - The obstacle must have an id and a box.
        """
        self.idx = idx
        self.box = box

In [9]:
def id_to_color(idx):
    # Random function to convert an id to a color
    blue = idx * 5 % 256
    green = idx * 36 % 256
    red = idx * 23 % 256
    return (red, green, blue)

In [10]:
def main():
    # assigning an id and draw a rectangle based on each id
    
    idx = 0
    obstacles = []
    result_images_2 = copy.deepcopy(dataset_images) # Copy the image without modifying the dataset
    for j, boxes in enumerate(result_boxes): # loop through all images
        for i, box in enumerate(boxes): # loop through all boxes
            obs = Obstacle(idx, box)
            left, top, width, height = box
            right = left + width
            bottom = top + height
            cv2.rectangle(result_images_2[j], (left, top), (right, bottom), id_to_color(idx+i), thickness=10)
            idx += 1
            obstacles.append(obs)
    return result_images_2

result_images_2 = main()

In [11]:
## Print the results
visualize_images(result_images_2)

Output hidden; open in https://colab.research.google.com to view.

The modification has been done!<p>

But as seen in the pictures, the colors are not kept constant along the images because tracking is not activated yet.

# 2 - Association

The next step is to match the detections from one frame to another frame and keep the color along the 9 images.<p>
It should be dynamic and work regardless of the number of images. 

![Texte alternatif…](https://miro.medium.com/proxy/0*yN9MllhmuglJORss.png)

### IOU COST

In [12]:
def convert_data(box):
    """
    Convert data from (x1,y1, w, h) to (x1,y1,x2,y2)
    """
    x1 = box[0]
    x2 = box[0] + box[2]
    y1 = box[1]
    y2 = box[1] + box[3]
    return x1, y1, x2, y2

def box_iou(box1, box2):
    """
    Computer Intersection Over Union cost
    """
    box1 = convert_data(box1)
    box2 = convert_data(box2)
    xA = max(box1[0], box2[0])
    yA = max(box1[1], box2[1])
    xB = min(box1[2], box2[2])
    yB = min(box1[3], box2[3])

    inter_area = max(0, xB - xA + 1) * max(0, yB - yA + 1) # abs((xi2 - xi1)*(yi2 - yi1))
    # Calculate the Union area by using Formula: Union(A,B) = A + B - Inter(A,B)
    box1_area = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1) # abs((box1[3] - box1[1])*(box1[2]- box1[0]))
    box2_area = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1) # abs((box2[3] - box2[1])*(box2[2]- box2[0]))
    union_area = (box1_area + box2_area) - inter_area
    # compute the IoU
    iou = inter_area/float(union_area)
    return iou

### Exponential, Linear, And IOU Costs

[This paper](https://arxiv.org/pdf/1709.03572.pdf) is used for exponential, linear, and IOU costs.

In [13]:
from math import sqrt, exp

def check_division_by_0(value, epsilon = 0.01):
  return (epsilon if value < epsilon else value)

def hungarian_cost(old_box, new_box, iou_thresh = 0.3, linear_thresh = 10000, exp_thresh = 0.5):
  w1 = 0.5
  w2 = 1.5
  (_, h, w, _) = np.array(dataset_images).shape
  
  # IOU_cost
  iou_cost = box_iou(old_box, new_box)

  # Sanchez-Matilla cost
  Q_dist = sqrt(pow(w, 2) + pow(h, 2))
  Q_shape = w * h
  value = sqrt(pow(old_box[0] - new_box[0], 2) + pow(old_box[1] - new_box[1], 2))
  value1 = sqrt(pow(old_box[2] - new_box[2], 2) + pow(old_box[3] - new_box[3], 2))
  distance_term = Q_dist / check_division_by_0(value)
  shape_term = Q_shape / check_division_by_0(value1)
  linear_cost = distance_term * shape_term

  # YUL cost
  a = (old_box[0] - new_box[0]) / check_division_by_0(old_box[2]) ** 2
  b = (old_box[1] - new_box[1]) / check_division_by_0(old_box[3]) ** 2
  ab = -(a + b) * w1
  c = abs(old_box[3] - new_box[3]) / (old_box[3] + new_box[3])
  d = abs(old_box[2] - new_box[2]) / (old_box[2] + old_box[2])
  cd = -(c + d) * w2
  exp_cost = exp(ab) * exp(cd)
  return (iou_cost if(iou_cost >= iou_thresh and linear_cost >= linear_thresh and exp_cost >= exp_thresh) else 0)

## The Hungarian Algorithm
The previous for tracking is used here to track bounding boxes!

The associate function takes two lists of boxes (time t-1 and time t) and outputs of it are the matches, the new detections, and the unmatched tracks.

In [14]:
from scipy.optimize import linear_sum_assignment

def associate(old_boxes, new_boxes):
    """
    old_boxes represents the former bounding boxes (at time 0)
    new_boxes represents the new bounding boxes (at time 1)
    Function goal: Define a Hungarian Matrix with IOU as a metric and return, for each box, an id
    RETURN: Matches, Unmatched Detections, Unmatched Trackers
    """
    # Define a new IOU Matrix with old and new boxes
    iou_matrix = np.zeros([len(old_boxes), len(new_boxes)], dtype = np.float32)

    # Loop through boxes and store the IOU value for each box
    for i, old_box in enumerate(old_boxes):
      for j, new_box in enumerate(new_boxes):
        iou_matrix[i][j] = hungarian_cost(old_box, new_box)

    # Call for the hungarian algorithm
    hungarian_row, hungarian_col = linear_sum_assignment(-iou_matrix)
    hungarian_matrix = np.array(list(zip(hungarian_row, hungarian_col)))

    # Create new unmatched lists for old and new boxes
    matches, unmatched_detections, unmatched_trackers = [], [], []

    # Loop through the  Hungarian matrix, if IOU of matched element is less than 0.3 then add to the unmatched list
    for h in hungarian_matrix:
      if(iou_matrix[h[0], h[1]] < 0.3):
        unmatched_trackers.append(old_boxes[h[0]])
        unmatched_detections.append(new_boxes[h[1]])
      else:
        matches.append(h.reshape(1, 2))

    if(len(matches) == 0):
      matches = np.empty((0, 2), dtype = int)
    else:
      matches = np.concatenate(matches, axis = 0)

    # Loop through old_boxes, if there is not any matched detection then add it to unmatched_old_boxes
    for t, trk in enumerate(old_boxes):
      if(t not in hungarian_matrix[:, 0]):
        unmatched_trackers.append(trk)

    # Loop through new_boxes, if there is not any matched tracking then add it to unmatched_new_boxes
    for d, det in enumerate(new_boxes):
      if(d not in hungarian_matrix[:, 0]):
        unmatched_detections.append(det)

    return matches, unmatched_detections, unmatched_trackers

## Main Loop

In [15]:
def main(input_image):
    """
    Receives an images
    Outputs the result image, and a list of obstacle objects 
    """
    
    global stored_obstacles # will be used to keep track of obstacles info
    global idx # will be used to keep track of id info

    # Run obstacle detection
    image = copy.deepcopy(input_image)
    _, out_boxes = yolo.inference(input_image)

    # First iteration: find obstacles and draw rectangles
    if(idx == 0):
      stored_obstacles = []
      for i, box in enumerate(out_boxes):
        obs = Obstacle(idx, box)
        left, top, right, bottom = convert_data(box) # Move to x1, x2, y1, y2
        cv2.rectangle(image, (left, top), (right, bottom), id_to_color(obs.idx), thickness = 10) # Draw the box on the image with id
        image = cv2.putText(image, str(obs.idx), (left - 10, top - 10), cv2.FONT_HERSHEY_PLAIN, 1, id_to_color(obs.idx), thickness = 4)
        stored_obstacles.append(obs) # Put every created obstacle in the final list
        idx += 1
      return image, stored_obstacles
    elif(idx != 0): # In this case, we already have obstacles from the previous frame, Now it is time to association and connecting
      # Before calling associate, a list of old obstacles should be created
      old_obstacles = [obs.box for obs in stored_obstacles]
      matches, unmatched_detections, unmatched_trackes = associate(old_obstacles, out_boxes) # Associate the obstacles
      new_obstacles = []

      # For every match, change the obstacle value
      # Assign the id to the matched id
      # Assign the box ot the new box

      for match in matches:
        obs = Obstacle(stored_obstacles[match[0]].idx, out_boxes[match[1]])
        new_obstacles.append(obs)

      # Loop through all unmatched detections and add these to obstacles
      for new_obs in unmatched_detections:
        idx += 1
        obs = Obstacle(idx, new_obs)
        new_obstacles.append(obs)

      # For every obstacle, it is needed to draw on the image and return it
      for i, obs in enumerate(new_obstacles):
        left, top, right, bottom = convert_data(obs.box)
        cv2.rectangle(image, (left, top), (right, bottom), id_to_color(obs.idx), thickness = 10)
        image = cv2.putText(image, str(obs.idx), (left - 10, top - 10), cv2.FONT_HERSHEY_PLAIN, 1, id_to_color(obs.idx), thickness = 4)
      stored_obstacles = copy.deepcopy(new_obstacles)
      return image, stored_obstacles

In [16]:
### Call the main loop

yolo = YOLO()
idx = 0

fig=plt.figure(figsize=(100,100))

result_images_3 = copy.deepcopy(dataset_images)

out_imgs = []

for i in range(len(result_images_3)):
    out_img, stored_obstacles = main(result_images_3[i])
    out_imgs.append(out_img)
    fig.add_subplot(1, len(result_images_3), i+1)
    plt.imshow(out_imgs[i])

plt.show()

Output hidden; open in https://colab.research.google.com to view.

# 3 - Improvements

### Using the Non Maxima Suppression formula

In order to avoid results like below picture, NMS is going to be used:<p>
![](https://user-images.githubusercontent.com/25801568/79720833-01a88180-82ea-11ea-993b-8accd6b7fcc1.png)

A Non-Maxima Suppression formula is used based on this page [page](https://www.pyimagesearch.com/2015/02/16/faster-non-maximum-suppression-python/).

In order to do this, we have to change (x1, y1, w, h) to (x1, y1, x2, y2).


In [17]:
os.chdir("/content/drive/My Drive/Colab Notebooks/Tracking/Main")
from yolo_nms import *

yolo = YOLO()
idx = 0

fig=plt.figure(figsize=(100,100))

result_images_3 = copy.deepcopy(dataset_images)

out_imgs = []

for i in range(len(result_images_3)):
    out_img, stored_obstacles = main(result_images_3[i])
    out_imgs.append(out_img)
    fig.add_subplot(1, len(result_images_3), i+1)
    plt.imshow(out_imgs[i])


Output hidden; open in https://colab.research.google.com to view.

### Using Age

Currently, the tracker is working very good! <p>
One thing that is not appropriate is that it relies solely on the detector.
If we miss the detection, we miss everything. <p>
Therefore, we need to add:
* False Positive
* False Negative<p>

A **false positive** means that an pobstacle is detected an obstacle, which it should not be detected.<p>
This can be solved by using a **MIN_HIT_STREAK** variable. If the detector detects something just for one time, it is not displayed. If it detects it more than once in a row, it is displayed.

A **false negative** means that an obstacle is not detected, which it should be detected.<p>
This can be solved by using a **MAX_AGE** variable. If an obstacle is suddently unmatched, we keep displaying it. If it is unmatched again for several times, it is removed.

In [18]:
MIN_HIT_STREAK = 1
MAX_UNMATCHED_AGE = 2

**Obstacle Class** <p>
Redefine the Obstacle class to include these values.
Every obstacle should have:
* an id
* a box
* an age (number of times matched)
* an unmatched frame number (number of times unmatched)

In [19]:
class Obstacle():
    def __init__(self, idx, box, age = 1, unmatched_age = 0):
        self.idx = idx
        self.box = box
        self.age = age
        self.unmatched_age = unmatched_age

In [20]:
def main(input_image):
    """
    Receives an images
    Outputs the result image, and a list of obstacle objects 
    """
    global stored_obstacles # Will be used to keep track of obstacles information
    global idx # Will be used to keep track of id information
    # Run obstacle detection
    image = copy.deepcopy(input_image)
    _, out_boxes = yolo.inference(input_image)
    
    # What we will do will be very similar but we have a second list of obstacles that answer to the conditions
    # On first iteration, we only create obstacles with age=1
    if (idx == 0):
        stored_obstacles = []
        for i, box in enumerate(out_boxes):
            obs = Obstacle(idx, box) # Create an obstacle with age=1
            stored_obstacles.append(obs)                
            idx +=1
        return image
    
    # On this case, if the obstacle has already been matched, we display it depending on the MIN_HIT_STREAK variable
    elif (idx != 0): # In case we already have obstacles from previous frame, work on association
        ## Before calling associate, we must create a list of old obstacles
        old_obstacles = [obs.box for obs in stored_obstacles] # Simply get the boxes
        matches, unmatched_detections, unmatched_tracks = associate(old_obstacles, out_boxes) # Associate the obstacles
        
        selected_obstacles = []
        # Loop through all matches and add these as obstacles
        new_obstacles = []
        for match in matches:
            obs = Obstacle(stored_obstacles[match[0]].idx, out_boxes[match[1]], stored_obstacles[match[0]].age +1) # Increase the age by 1
            new_obstacles.append(obs)
            if obs.age >= MIN_HIT_STREAK:
                selected_obstacles.append(obs)
        
        # Loop through all unmatched detections and add these as obstacles
        for new_obs in unmatched_detections:
            idx +=1
            obs = Obstacle(idx, new_obs)
            new_obstacles.append(obs)
            if obs.age >= MIN_HIT_STREAK:
                selected_obstacles.append(obs)

        for i, old_obs in enumerate(unmatched_tracks):
            if stored_obstacles[i].box == old_obs:
                obs = stored_obstacles[i] 
                obs.unmatched_age +=1
                if obs.unmatched_age <= MAX_UNMATCHED_AGE:
                    selected_obstacles.append(obs)

        # Draw on selected obstacles only
        for i, obs in enumerate(selected_obstacles):
            left, top, right, bottom = convert_data(obs.box)
            cv2.rectangle(image, (left, top), (right, bottom), id_to_color(obs.idx), thickness=10)
            image = cv2.putText(image, str(obs.idx),(left,top),cv2.FONT_HERSHEY_SIMPLEX, 1,id_to_color(obs.idx),thickness=4)                

        stored_obstacles = copy.deepcopy(new_obstacles)
        return image

In [21]:
yolo = YOLO()
idx = 0

fig=plt.figure(figsize=(100,100))

result_images_3 = copy.deepcopy(dataset_images)

out_imgs = []

for i in range(len(result_images_3)):
    out_img = main(result_images_3[i])
    out_imgs.append(out_img)
    fig.add_subplot(1, len(result_images_3), i+1)
    plt.imshow(out_imgs[i])

plt.show()

Output hidden; open in https://colab.research.google.com to view.

# 4 - Kalman Filter

Now, It is time to introduce Kalman Filter to this project. Kalman Filters will help predict the future position of a bounding box, so that the association will always match in the future.

state of the Kalman Filter consistes of 4 variables: x, y, w, h. <p>
These are the values returned by the YOLO algorithm.

The process will be the following
*   Detect bounding boxes
*   Associate it with the previous predictions
*   Predict the next position of each box
<p>
The association is made with the prediction from t-1.

In [23]:
!pip install filterpy

     |████████████████████████████████| 184kB 16.3MB/s eta 0:00:01
  Created wheel for filterpy: filename=filterpy-1.4.5-cp36-none-any.whl size=110453 sha256=3b5737a4f10a1c58b3a8f66480225ceae188ec853140c7f2930993c13f5dc56d
  Stored in directory: /root/.cache/pip/wheels/c3/0c/dd/e92392c3f38a41371602d99fc77d6c1d42aadbf0c6afccdd02
Successfully built filterpy


In [24]:
from filterpy.kalman import KalmanFilter
from scipy.linalg import block_diag
from filterpy.common import Q_discrete_white_noise
import time

def FourDimensionsKF(R_std = 10, Q_std = 0.01):
    kf = KalmanFilter(dim_x = 8, dim_z = 4)
    kf.F = np.array([[1, 1, 0, 0, 0, 0, 0, 0],
                     [0, 1, 0, 0, 0, 0, 0, 0],
                     [0, 0, 1, 1, 0, 0, 0, 0],
                     [0, 0, 0, 1, 0, 0, 0, 0],
                     [0, 0, 0, 0, 1, 1, 0, 0],
                     [0, 0, 0, 0, 0, 1, 0, 0],
                     [0, 0, 0, 0, 0, 0, 1, 1],
                     [0, 0, 0, 0, 0, 0, 0, 1]])

    kf.P *= 1000
    kf.R[2:, 2:] *= R_std
    kf.Q[-1, -1] *= Q_std
    kf.Q[4:, 4:] *= Q_std
    return kf

In [25]:
class Obstacle():
    def __init__(self, idx, box, time, age = 1, unmatched_age = 0):
        self.idx = idx
        self.box = box
        self.age = age
        self.unmatched_age = unmatched_age
        self.time = time
        self.kf = FourDimensionsKF()
        self.kf.x = np.array([box[0], 0, box[1], 0, box[2], 0, box[3], 0])
        self.kf.H = np.array([[1, 0, 0, 0, 0, 0, 0, 0],
                             [0, 0, 1, 0, 0, 0, 0, 0],
                             [0, 0, 0, 0, 1, 0, 0, 0],
                             [0, 0, 0, 0, 0, 0, 1, 0]])

In [26]:
def get_obs_from_mean(mean):
    return [mean[0], mean[2], mean[4], mean[6]]

In [27]:
def return_F_with_dt(dt):
    dt = 1./25. #VIDEO MODE
    F = np.array([[1,dt, 0,  0, 0, 0, 0, 0],
                  [0, 1, 0,  0, 0, 0, 0, 0],
                  [0, 0, 1, dt, 0, 0, 0, 0],
                  [0, 0, 0,  1, 0, 0, 0, 0],
                  [0, 0, 0, 0, 1, dt, 0, 0],
                  [0, 0, 0, 0, 0,  1, 0, 0],
                  [0, 0, 0, 0, 0, 0, 1, dt],
                  [0, 0, 0, 0, 0, 0, 0,  1]])
    return F

In [28]:
def main(input_image):
    """
    Receives an images
    Outputs the result image, and a list of obstacle objects 
    """
    global stored_obstacles
    global idx
    global yolo
    
    image = copy.deepcopy(input_image)
    _, out_boxes = yolo.inference(input_image)
    current_time = time.time()
    
    # First Detection, Initialize a Kalman Filter per Bounding Box
    if (idx == 0):
        stored_obstacles = []
        for i, box in enumerate(out_boxes):
            obs = Obstacle(idx, box, current_time)
            stored_obstacles.append(obs)
            idx +=1
        return input_image
    
    # Not First Detection
    elif (idx != 0):                
        # Match between old obstacles and new using Hungarian Algorithm
        old_boxes = [obs.box for obs in stored_obstacles]
        matches, unmatched_detections, unmatched_tracks = associate(old_boxes, out_boxes)

        selected_obstacles = []
        new_obstacles = []

        # For Matched Obstacles, Update & Predict the next position; store in Box for future match
        for match in matches:
            obs = stored_obstacles[match[0]] # Take the former obstacle and its ID
            obs.age += 1 # Increment the age by 1
            
            # Update
            measurement = out_boxes[match[1]]         
            obs.kf.update(np.array(measurement))

            # Prediction
            F = return_F_with_dt(current_time - obs.time)
            obs.kf.F = F
            obs.kf.predict()
            obs.time = current_time
            obs.box = get_obs_from_mean(obs.kf.x)
            
            new_obstacles.append(obs)
            if obs.age >= MIN_HIT_STREAK:
                selected_obstacles.append(obs)

        # For Unmatched Detections, the same as idx = 0
        for new_obs in unmatched_detections:
            idx += 1
            obs = Obstacle(idx, new_obs, current_time)
            new_obstacles.append(obs)
            if obs.age >= MIN_HIT_STREAK:
                selected_obstacles.append(obs)

        # For Unmatched Tracks, Predict using dt
        for i, old_obs in enumerate(unmatched_tracks):
            if stored_obstacles[i].box == old_obs:
                obs = stored_obstacles[i]
                F = return_F_with_dt(current_time - obs.time)
                obs.time = current_time
                obs.kf.F = F
                obs.kf.predict()
                obs.box = get_obs_from_mean(obs.kf.x)
                obs.unmatched_age += 1
                if obs.unmatched_age <= MAX_UNMATCHED_AGE:
                    selected_obstacles.append(obs)

        # Draw on selected obstacles only
        for i, obs in enumerate(selected_obstacles):
            left, top, right, bottom = convert_data(obs.box)
            cv2.rectangle(image, (int(left), int(top)), (int(right), int(bottom)), id_to_color(obs.idx), thickness = 10)
            image = cv2.putText(image, str(obs.idx),(int(left),int(top)),cv2.FONT_HERSHEY_SIMPLEX, 1,id_to_color(obs.idx),thickness = 4)                
        stored_obstacles = copy.deepcopy(new_obstacles)
        return image

In [29]:
yolo = YOLO()

idx = 0

fig=plt.figure(figsize=(100,100))

result_images_3 = copy.deepcopy(dataset_images)

out_imgs = []

for i in range(len(result_images_3)):
    out_img = main(result_images_3[i])
    out_imgs.append(out_img)
    fig.add_subplot(1, len(result_images_3), i + 1)
    plt.imshow(out_imgs[i])

plt.show()

Output hidden; open in https://colab.research.google.com to view.

# Video

In [30]:
from moviepy.editor import VideoFileClip
idx = 0
detector = YOLO()
video_file = "/content/drive/My Drive/Colab Notebooks/Tracking/Main/Images/MOT16-14-raw.mp4" #25 FPS
clip = VideoFileClip(video_file).subclip(0,5)
white_clip = clip.fl_image(main)
%time white_clip.write_videofile("/content/drive/My Drive/Colab Notebooks/Tracking/Main/Images/movie_track_kf_out.mp4",audio=False)

[MoviePy] >>>> Building video /content/drive/My Drive/Colab Notebooks/Tracking/Main/Images/movie_track_kf_out.mp4
[MoviePy] Writing video /content/drive/My Drive/Colab Notebooks/Tracking/Main/Images/movie_track_kf_out.mp4


 99%|█████████▉| 125/126 [02:15<00:01,  1.09s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /content/drive/My Drive/Colab Notebooks/Tracking/Main/Images/movie_track_kf_out.mp4 

CPU times: user 3min 58s, sys: 8.16 s, total: 4min 7s
Wall time: 2min 17s
